<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="#数据准备" data-toc-modified-id="数据准备-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>数据准备</a></span></li><li><span><a href="#获取评分详情" data-toc-modified-id="获取评分详情-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>获取评分详情</a></span></li></ul></div>

## Setup

In [102]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from datetime import datetime
import pprint

In [103]:
es = Elasticsearch()

## 数据准备

In [104]:
test_score_index = "test-index"

In [148]:
my_data_settting = {
    "settings": {
        "analysis": {
            "analyzer": {
                "my_analyzer": {
                    "tokenizer": "standard",
                    "char_filter": [
                        "my_char_filter"
                    ]
                } 
            },
            "char_filter": {
                "my_char_filter": {
                    "type": "mapping",
                    "mappings": [
                        "- => _hyphen_",
                    ]
                }
            }
        }
    },
    "mappings": {
        "my_type": {
            "properties": {
                "cmd_format_t": {
                    "type": "text",
                    "search_analyzer": "my_analyzer",
                    "analyzer": "my_analyzer",
                    "norms": { "enabled": False }
                }
            }
        }
    }
}

In [149]:
es.indices.delete(index=test_score_index)
es.indices.create(index=test_score_index, body=my_data_settting)

{'acknowledged': True}

{'acknowledged': True, 'index': 'test-index', 'shards_acknowledged': True}

In [150]:
doc1 = {
    'author': 'tangting',
    'cmd_format_t': 'This is a brown dog a b',
    'len': 3
}
doc2 = {
    'author': 'tangting',
    'cmd_format_t': 'This is a brown dog a asf sdfsdf s sdfsdfsdfsf  sdf s s s s s',
    'len': 4
}
doc3 = {
    'author': 'tangting',
    'cmd_format_t': 'This is a brown dog a b',
    'len': 5
} 
actions = [{ "_index": test_score_index, "_type": "my_type", "_id": idx, "_source": value} for idx, value in enumerate([doc1, doc2, doc3])]
helpers.bulk(es, actions)  

(3, [])

In [151]:
es.search(index=test_score_index, body={"query": {"match_all": {}}})

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [], 'max_score': None, 'total': 0},
 'timed_out': False,
 'took': 0}

## 获取评分详情

In [152]:
es.search(index=test_score_index, explain=False, body={
    "query": {
        "match_phrase": {
            "cmd_format_t": {
                "query": "brown  a dog", "slop": 30
            }
        }
    },
    "highlight": {
        "fields" : {
            "cmd_format_t" : {}
        }
    }
})

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '0',
    '_index': 'test-index',
    '_score': 0.6781078,
    '_source': {'author': 'tangting',
     'cmd_format_t': 'This is a brown dog a b',
     'len': 3},
    '_type': 'my_type',
    'highlight': {'cmd_format_t': ['This is <em>a</em> <em>brown</em> <em>dog</em> <em>a</em> b']}},
   {'_id': '2',
    '_index': 'test-index',
    '_score': 0.6781078,
    '_source': {'author': 'tangting',
     'cmd_format_t': 'This is a brown dog a b',
     'len': 5},
    '_type': 'my_type',
    'highlight': {'cmd_format_t': ['This is <em>a</em> <em>brown</em> <em>dog</em> <em>a</em> b']}},
   {'_id': '1',
    '_index': 'test-index',
    '_score': 0.6781078,
    '_source': {'author': 'tangting',
     'cmd_format_t': 'This is a brown dog a asf sdfsdf s sdfsdfsdfsf  sdf s s s s s',
     'len': 4},
    '_type': 'my_type',
    'highlight': {'cmd_format_t': ['This is <em>a</em> <em>brown</em> <em>dog</em> <em>a<

In [161]:
es.search(index=test_score_index, explain=True, body={
    "query": {
        "function_score": {
            "query": {
                "match": {
                    "cmd_format_t": {
                        "query": "brown dog"
                    }
                }
            },
            "field_value_factor": {
                "field": "len",
                "modifier": "reciprocal",
                "factor": 0.9
            },
            "boost_mode": "multiply"
        }
    },
    "highlight": {
        "fields" : {
            "cmd_format_t" : {}
        }
    }
})

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_explanation': {'description': 'function score, product of:',
     'details': [{'description': 'sum of:',
       'details': [{'description': 'weight(cmd_format_t:brown in 0) [PerFieldSimilarity], result of:',
         'details': [{'description': 'score(doc=0,freq=1.0 = termFreq=1.0\n), product of:',
           'details': [{'description': 'idf, computed as log(1 + (docCount - docFreq + 0.5) / (docFreq + 0.5)) from:',
             'details': [{'description': 'docFreq',
               'details': [],
               'value': 1.0},
              {'description': 'docCount', 'details': [], 'value': 1.0}],
             'value': 0.2876821},
            {'description': 'tfNorm, computed as (freq * (k1 + 1)) / (freq + k1) from:',
             'details': [{'description': 'termFreq=1.0',
               'details': [],
               'value': 1.0},
              {'description': 'parameter k1', 'details': [], 'va

In [156]:
0.5753642 * 5

2.876821